# Mapping of US Census tracts to US Postal Service ZIP codes and US Counties

**[Work in progress]**

This notebook creates .csv files with mappings between US Postal Service ZIP codes and US Census Tracts provide by the U.S. Department of Housing and Urban Development (HUD), and mappings between US Counties and US Census Tracts provided by the US Census American Community Survey 5-Year Data (2009-2018).

US County to US Census Tract mappings are

Reference: Wilson, Ron and Din, Alexander, 2018. “Understanding and Enhancing the U.S. Department of Housing and Urban Development’s ZIP Code Crosswalk Files”, [Cityscape: A Journal of Policy Development and Research, Volume 20 Number 2, 277 – 294.]( https://www.huduser.gov/portal/periodicals/cityscpe/vol20num2/ch16.pdf)

Data sources: 

[HUD USPS ZIP CODE CROSSWALK FILES](https://www.huduser.gov/portal/datasets/usps_crosswalk.html)

[American Community Survey 5-Year Data (2009-2018)](https://www.census.gov/data/developers/data-sets/acs-5year.html)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0/import


### 2020Q1 USPS ZIP code to US County mappings

In [4]:
zip_to_county_url = 'https://www.huduser.gov/portal/datasets/usps/ZIP_COUNTY_032020.xlsx'

In [5]:
zip_to_county = pd.read_excel(zip_to_county_url, dtype='str')

In [6]:
zip_to_county.head()

,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,00501,36103,0,1,0,1
1,00601,72113,0.16072360410506176,0.20098039215686275,0.12883435582822086,0.1625
2,00601,72001,0.8392763958949383,0.7990196078431373,0.8711656441717791,0.8375
3,00602,72003,1,0.9987995198079231,1,0.9999193483345431
4,00602,72005,0,0.0012004801920768306,0,8.065166545689168e-05


In [7]:
zip_to_county.rename(columns={'COUNTY': 'countyFips', 'ZIP': 'zip', 'RES_RATIO': 'resRatio'}, inplace=True)
zip_to_county.rename(columns={'BUS_RATIO': 'busRatio', 'OTH_RATIO': 'othRatio', 'TOT_RATIO': 'totRatio'}, inplace=True)

In [8]:
zip_to_county['stateFips'] = zip_to_county['countyFips'].str[:2]
zip_to_county['countyFips'] = zip_to_county['countyFips'].str[2:]

In [9]:
zip_to_county.query("zip == '92301'")

,zip,countyFips,resRatio,busRatio,othRatio,totRatio,stateFips
50514,92301,037,7.441029838529652e-05,0,0,6.807351940095303e-05,06
50515,92301,071,0.9999255897016147,1,1,0.999931926480599,06


Example of one Zip area that maps to multiple counties.

In [10]:
zip_to_county.query("zip == '21771'")

,zip,countyFips,resRatio,busRatio,othRatio,totRatio,stateFips
9710,21771,027,0.10808996307485734,0.035147392290249435,0.024193548387096774,0.10230614455966569,24
9711,21771,013,0.46273917421953675,0.8140589569160998,0.7580645161290323,0.48955270082030644,24
9712,21771,031,0.011581067472306143,0.003401360544217687,0,0.010911623587679926,24
9713,21771,021,0.41758979523329975,0.1473922902494331,0.21774193548387097,0.3972295310323479,24


In [10]:
zip_to_county.to_csv(NEO4J_IMPORT / "00m-USHUDCrosswalkZipToCounty2020Q1.csv", index=False)

### 2020Q1 USPS ZIP to US Census Tract mappings

In [11]:
zip_to_tract_url = 'https://www.huduser.gov/portal/datasets/usps/ZIP_TRACT_032020.xlsx'

In [12]:
zip_to_tract = pd.read_excel(zip_to_tract_url, dtype='str')

In [13]:
zip_to_tract.rename(columns={'TRACT': 'tract', 'ZIP': 'zip', 'RES_RATIO': 'resRatio'}, inplace=True)
zip_to_tract.rename(columns={'BUS_RATIO': 'busRatio', 'OTH_RATIO': 'othRatio', 'TOT_RATIO': 'totRatio'}, inplace=True)

In [14]:
zip_to_tract.head()

,zip,tract,resRatio,busRatio,othRatio,totRatio
0,00501,36103158607,0,1,0,1
1,00601,72001956800,0.01481610597873453,0.01256281407035176,0.0440251572327044,0.015411503018748014
2,00601,72113071700,0.16105978734530244,0.20603015075376885,0.1320754716981132,0.16317127422942485
3,00601,72001956600,0.17291267212829003,0.3542713567839196,0.3584905660377358,0.1890689545598983
4,00601,72001956700,0.651211434547673,0.4271356783919598,0.46540880503144655,0.6323482681919288


In [15]:
zip_to_tract.query("zip == '92121'")

,zip,tract,resRatio,busRatio,othRatio,totRatio
156595,92121,06073008346,0.5238353196099675,0.2380638788277906,0.2620481927710843,0.29728601252609604
156596,92121,06073008350,0,0.49078037537043134,0.4716867469879518,0.39290187891440503
156597,92121,06073009400,0,0.00032927230819888045,0,0.00020876826722338206
156598,92121,06073008339,0.4761646803900325,0.2708264734935792,0.26626506024096386,0.3096033402922756


In [16]:
zip_to_tract.to_csv(NEO4J_IMPORT / "00m-USHUDCrosswalkZipToTract2020Q1.csv", index=False)

In [17]:
len(zip_to_tract['tract'].unique())

73467

### Census Tract to County mappings

In [18]:
url_county = f'https://api.census.gov/data/2018/acs/acs5/profile?get=NAME&for=county:*'

In [19]:
df = pd.read_json(url_county, dtype='str')
# skip first row of labels
df = df[1:].copy() 
columns = []
columns.append('Name')
columns.append('stateFips')
columns.append('countyFips')
df.columns = columns

In [20]:
# Remove Puerto Rico (stateFips = 72) to limit data to US States.
# GeoNames represents Puerto Rico as a country, rather than a US state.
# TODO how to handle data for Puerto Rico?
df.query("stateFips != '72'", inplace=True) 

In [21]:
stateFips = list(df['stateFips'].unique())
stateFips.sort()
print(stateFips)

['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']


In [22]:
def get_tract_data(state):
    url_tract = f'https://api.census.gov/data/2018/acs/acs5/profile?get=NAME&for=tract:*&in=state:{state}'
    df = pd.read_json(url_tract, dtype='str')
    # skip first row of labels
    df = df[1:].copy()
    # Add column names
    columns = []
    columns.append('NAME')
    columns.append('stateFips')
    columns.append('countyFips')
    columns.append('tract')
    df.columns = columns
    return df

In [23]:
tract_to_county = pd.concat((get_tract_data(state) for state in stateFips))

In [24]:
tract_to_county['tract'] = tract_to_county['stateFips'] + tract_to_county['countyFips'] + tract_to_county['tract']

In [25]:
tract_to_county.head()

,NAME,stateFips,countyFips,tract
1,"Census Tract 57.01, Jefferson County, Alabama",01,073,01073005701
2,"Census Tract 107.04, Jefferson County, Alabama",01,073,01073010704
3,"Census Tract 129.08, Jefferson County, Alabama",01,073,01073012908
4,"Census Tract 53.02, Jefferson County, Alabama",01,073,01073005302
5,"Census Tract 111.07, Jefferson County, Alabama",01,073,01073011107


In [26]:
tract_to_county.to_csv(NEO4J_IMPORT / "00m-USHUDCrosswalkTractToCounty.csv", index=False)

#### Data checks

##### Tracts in San Diego county

In [27]:
tract_to_county[(tract_to_county['stateFips'] == '06') & (tract_to_county['countyFips'] == '073')].head()

,NAME,stateFips,countyFips,tract
56,"Census Tract 83.24, San Diego County, California",06,073,06073008324
57,"Census Tract 83.39, San Diego County, California",06,073,06073008339
58,"Census Tract 83.47, San Diego County, California",06,073,06073008347
59,"Census Tract 83.54, San Diego County, California",06,073,06073008354
60,"Census Tract 85.05, San Diego County, California",06,073,06073008505


##### Number of counties in US states

In [28]:
tract_to_county['f'] = tract_to_county['countyFips'] + tract_to_county['stateFips']
len(tract_to_county['f'].unique())

3142

##### Number of tracts in US states

In [29]:
len(tract_to_county['tract'].unique())

73056

##### Tracts in common between data from HUD vs ACS 2018 5-year

TODO Where does this difference come from??? None of the HUD tracts (different dates) match the ACS 2018 5-year data exactly.

In [30]:
intersection = zip_to_tract.merge(tract_to_county, left_on='tract', right_on='tract', how='inner')

In [31]:
len(intersection['tract'].unique())

72616